<a href="https://colab.research.google.com/github/feliciahf/NLP-Project/blob/main/NB_Scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multinomial Naive Bayes

## Importing Data

In [1]:
# mount Google Drive
from google.colab import  drive
drive.mount('/drive')

Mounted at /drive


In [2]:
# import file from Google Drive
import pandas as pd
df = pd.read_csv('/drive/My Drive/book32listing.csv',encoding='latin1', header=None)

In [3]:
# drop columns that are not needed
#df = pd.read_csv("book32listing.csv", encoding='latin1', header=None)
df1 = df[[3,6,5]] # only columns with titles and genres
df1.columns = ['title', 'genre', 'label']
print(df1)

                                                    title      genre  label
0                         Mom's Family Wall Calendar 2016  Calendars      3
1                         Doug the Pug 2016 Wall Calendar  Calendars      3
2       Moleskine 2016 Weekly Notebook, 12M, Large, Bl...  Calendars      3
3                 365 Cats Color Page-A-Day Calendar 2016  Calendars      3
4                    Sierra Club Engagement Calendar 2016  Calendars      3
...                                                   ...        ...    ...
207567  ADC the Map People Washington D.C.: Street Map...     Travel     29
207568  Washington, D.C., Then and Now: 69 Sites Photo...     Travel     29
207569  The Unofficial Guide to Washington, D.C. (Unof...     Travel     29
207570      Washington, D.C. For Dummies (Dummies Travel)     Travel     29
207571  Fodor's Where to Weekend Around Boston, 1st Ed...     Travel     29

[207572 rows x 3 columns]


## Preprocessing

In [4]:
# case collapsing
df1['title'] = df1.title.map(lambda x: x.lower())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
# remove punctuation
df1['title'] = df1.title.str.replace('[^\w\s]', '')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
# tokenization
df1['title'] = df1['title'].apply(nltk.word_tokenize)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
# transform data into occurrences
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings
df1['title'] = df1['title'].apply(lambda x: ' '.join(x))

count_vect = CountVectorizer()
counts = count_vect.fit_transform(df1['title'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
# tf-idf
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)

counts = transformer.transform(counts)

## Training NB Model

In [10]:
# split data into train (80%) and test (20%)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, df1['label'], test_size=0.2, random_state=69)

In [11]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)

## Evaluating NB Model

In [12]:
# accuracy
import numpy as np

predicted = model.predict(X_test)

print(np.mean(predicted == y_test))

0.544285198121161


In [13]:
# compute overall accuracy, precision, recall, f1 scores
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print('Accuracy: ', accuracy_score(y_test, predicted))
print('Precision: ', precision_score(y_test, predicted, average='weighted', zero_division=1))
print('Recall: ', recall_score(y_test, predicted, average='weighted', zero_division=1))
print('F1:', f1_score(y_test, predicted, average='weighted'))

Accuracy:  0.544285198121161
Precision:  0.6447742332900409
Recall:  0.544285198121161
F1: 0.5061149240160508


In [14]:
# compute accuracy, precision, recall, f1 scores by genre

from sklearn.metrics import precision_recall_fscore_support as score

# precision, recall, fscore, support separated by genre
precision, recall, fscore, support = score(y_test, predicted)

df_acc = pd.DataFrame()
df_acc['precision']=pd.Series(precision)
df_acc['recall']=pd.Series(recall)
df_acc['fscore']=pd.Series(fscore)
df_acc['support']=pd.Series(support)

print(df_acc)
# indexing corresponds to genre ID in original data

    precision    recall    fscore  support
0    0.763920  0.262232  0.390438     1308
1    0.538462  0.025090  0.047945      837
2    0.526874  0.748744  0.618514     1990
3    0.913151  0.736000  0.815061      500
4    0.342926  0.774088  0.475294     2740
5    0.935780  0.170569  0.288543      598
6    0.819454  0.724181  0.768877     1617
7    0.829894  0.763961  0.795564     1737
8    0.631603  0.716334  0.671305     1953
9    0.527482  0.704272  0.603189     1826
10   0.949153  0.115464  0.205882      485
11   0.423853  0.770964  0.546988     2397
12   0.600439  0.406994  0.485144     1344
13   0.708197  0.320713  0.441492     1347
14   0.775742  0.608904  0.682272     1460
15   0.520197  0.341748  0.412500     1545
16   0.603854  0.794366  0.686131     2485
17   0.933333  0.032634  0.063063      429
18   1.000000  0.011765  0.023256      510
19   1.000000  0.031546  0.061162      634
20   0.786517  0.106061  0.186916      660
21   0.675479  0.503397  0.576878     1472
22   0.8724

In [15]:
# confusion matrix
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predicted))

[[ 343    0   60 ...  277    0    0]
 [   7   21   30 ...  265    0    0]
 [   5    0 1490 ...  101    0    0]
 ...
 [   2    0   17 ... 3426    0    0]
 [   2    0   13 ...   56    1    0]
 [   0    0   88 ...   15    0    6]]


In [16]:
# examine class distribution
y_test.value_counts()

29    3663
4     2740
16    2485
11    2397
2     1990
8     1953
23    1858
9     1826
7     1737
6     1617
15    1545
27    1488
21    1472
14    1460
13    1347
12    1344
0     1308
26    1232
22     893
1      837
24     733
20     660
19     634
28     609
5      598
25     565
18     510
3      500
10     485
17     429
31     326
30     274
Name: label, dtype: int64